In [3]:
import requests
from bs4 import BeautifulSoup
import re
import sys
import argparse
# from selenium import webdriver
from io import BytesIO

try:
    import pytesseract
except:
    !pip install pytesseract  
    import pytesseract

try:
    from PIL import Image
except:
    !pip install PIL #manejo de imagines
    from PIL import Image

try:
    from selenium.webdriver import Chrome
except:
    !pip install selenium
    from selenium.webdriver import Chrome

pytesseract.pytesseract.tesseract_cmd = 'C:\Program Files\Tesseract-OCR\\tesseract.exe'  #aqui le vamos a mandar la direccion del ejecutable de teserac

  Created wheel for pytesseract: filename=pytesseract-0.3.0-py2.py3-none-any.whl size=20974 sha256=52df05378d0a57ca7d1b757877b17c9acc5b256acffb96062f798258f6723b3d
  Stored in directory: C:\Users\DMC\AppData\Local\pip\Cache\wheels\78\c9\ac\4cb76bd547f9970070522439e5203ba7926c5c5c4f131583ea
Successfully built pytesseract


In [6]:
driver = Chrome('chromedriver.exe')  #chromedriver_win32/chromedriver.exe lo primero es la carpeta
link = 'http://www.sunat.gob.pe/cl-ti-itmrconsruc/FrameCriterioBusquedaMovil.jsp'
driver.get(link)
# levantar en Home el chromedriver
# el resultado es que me abre la pagina de sunat
# 

In [22]:
driver.implicitly_wait(10) # este es el tiempo maximo que debo esperar 10 seg, buscar un elemento x nombre
element = driver.find_element_by_name('imagen') # aqui tenemos el elemento seleccionado osea el captcha

pos = element.location #guardamos location y luego size
size = element.size
png = driver.get_screenshot_as_png() #guarda un screenshot del navegador

img = Image.open(BytesIO(png)) #  
left = pos['x']
top = pos['y']
right = pos['x'] + size['width']
bottom = pos['y'] + size['height']
img = img.crop((left, top, right, bottom)) # se le da las 4 esquinas ya cortadas, aqui solo queda la imagen del captcha

captcha = pytesseract.image_to_string(img) # se va almacenar en el captcha

In [23]:
captcha

'WRAK'

In [24]:
ruc = '10749304389'
#ruc = '10107224527'
# con estas 3 lineas estamos llenado el formulario
driver.find_element_by_id('txtRuc').send_keys(ruc)  
driver.find_element_by_id('txtCodigo').send_keys(captcha)
driver.find_element_by_id('btnAceptar').click() 
#FUNCION .click (tercer linea)
#driver.implicitly_wait(10) # demora en cargar los resultados
#html = driver.find_element_by_class_name('list-group').get_attribute('innerHTML') #

In [25]:
driver.implicitly_wait(10) # demora en cargar los resultados
html = driver.find_element_by_class_name('list-group').get_attribute('innerHTML') #aqui es scrpear sobre codigo html
# lo guardamos en una variable html
# 

In [26]:
content = BeautifulSoup(html, 'html.parser')

In [27]:
content


<!-- Inicio filas de datos -->
<div class="list-group-item">
<div class="row">
<div class="col-sm-5">
<h4 class="list-group-item-heading">RUC:</h4>
</div>
<div class="col-sm-7">
<h4 class="list-group-item-heading">10749304389 - TUYA RODRIGUEZ PAULO CESAR</h4>
</div>
</div>
</div>
<div class="list-group-item">
<div class="row">
<div class="col-sm-5">
<h4 class="list-group-item-heading">Tipo Contribuyente:</h4>
</div>
<div class="col-sm-7">
<p class="list-group-item-text">PERSONA NATURAL SIN NEGOCIO</p>
</div>
</div>
</div>
<div class="list-group-item">
<div class="row">
<div class="col-sm-5">
<h4 class="list-group-item-heading">Tipo de Documento:</h4>
</div>
<div class="col-sm-7">
<p class="list-group-item-text">DNI  74930438 
						            
						            	 - TUYA RODRIGUEZ, PAULO CESAR
						            	
						            </p>
</div>
</div>
</div>
<div class="list-group-item">
<div class="row">
<div class="col-sm-5">
<h4 class="list-group-item-heading">Nombre Comercial:</h4>


In [28]:
inforuc = {}
# inforuc es el direcccionario vacio q se va a llenar
info = content.select('.list-group-item')
# el select por defecto devuelve todo lo q encuentre, q encuentre todos los list group item
for elem in info:
    key = elem.find(class_='col-sm-5').find('h4').text[:-1].strip() # [:-1] para eliminar los dos puntos, find h4 es otra etiqueta dentro. text=nos devuelve el texto dentro de la etiqueta, -1
    data = elem.find(class_='col-sm-7').text.strip() #
    inforuc[key] = data

In [29]:
inforuc

{'RUC': '10749304389 - TUYA RODRIGUEZ PAULO CESAR',
 'Tipo Contribuyente': 'PERSONA NATURAL SIN NEGOCIO',
 'Tipo de Documento': 'DNI  74930438 \n\t\t\t\t\t\t            \n\t\t\t\t\t\t            \t - TUYA RODRIGUEZ, PAULO CESAR',
 'Nombre Comercial': '-',
 'Fecha de Inscripción': '17/05/2018',
 'Estado': 'ACTIVO',
 'Condición': 'HABIDO',
 'Domicilio Fiscal': '-',
 'Actividad(es) Económica(s)': 'Principal        - CIIU 93098 - OTRAS ACTIVID.DE TIPO SERVICIO NCP',
 'Comprobantes de Pago c/aut. de impresión (F. 806 u 816)': 'NINGUNO',
 'Sistema de Emisión Electrónica': 'RECIBOS POR HONORARIOS     AFILIADO DESDE 21/05/2018',
 'Afiliado al PLE desde': '-',
 'Padrones': 'NINGUNO'}

In [30]:
driver.find_element_by_class_name('btnNuevaConsulta').click() # es para devolver

In [31]:
driver.quit()